In [2]:
import glob
import nbformat
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
import platform

In [ ]:
fpaths = glob.glob('../../accy575-sp-2023/02-pcard-submissions/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

In [ ]:
fpaths = glob.glob('../submissions/exercise-05/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

In [3]:
fpaths = glob.glob('test-notebooks/exercise-05/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

['test-notebooks/exercise-05\\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb',
 'test-notebooks/exercise-05\\esparzajoel_51499_7451315_Copy_of_exercise_05_pandas_filtering_sorting.ipynb',
 'test-notebooks/exercise-05\\tabornatalie_8175_7404582_exercise_05_pandas_filtering_sorting.ipynb']

In [20]:
import lambdagrader

graded_results = []

for notebook_path in fpaths:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        test_cases_hash = lambdagrader.get_test_cases_hash(nb)
        
        for cell in nb.cells:
            test_case_metadata = lambdagrader.extract_test_case_metadata_from_cell(cell.source)

            if test_case_metadata:
                cell.source = lambdagrader.convert_to_grader_code(cell.source)

        lambdagrader.add_grader_scripts(nb)
        
        print(f'Grading {notebook_path}')
        
        ep = ExecutePreprocessor(
            timeout=1800,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        
        # save graded notebook
        converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        
        # running the notebook will store the graded result to a JSON file
        # rename graded result JSON file
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        
        # read graded result to generate a summary
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
        
        # add filename
        # we add it here instead of trying to add it within the Jupyter notebook
        # because it is tricky to grab the current file name inside a Jupyter kernel
        graded_result['filename'] = Path(notebook_path).name
        
        # MD5 hash of test cases code
        # this helps us to identify any potential cases
        # where a learner has modified or deleted the test cases code cell
        graded_result['test_cases_hash'] = test_cases_hash
        
        # store Python version and platform used to run the notebook
        graded_result['grader_python_version'] = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
        graded_result['grader_platform'] = platform.platform()

        # save updated JSON to file
        with open(graded_result_json_path, 'w') as f:
            json.dump(graded_result, f, indent=2)
           
        # extract user code to a Python file
        extracted_user_code = lambdagrader.extract_user_code_from_notebook(nb)
        extracted_code_path = notebook_path.replace('.ipynb', '_user_code.py')
        with open(extracted_code_path, "w") as f:
            f.write(extracted_user_code)
            
        # clean up notebook
        lambdagrader.remove_grader_scripts(nb)
        lambdagrader.add_graded_result(nb, graded_result)
        
        # store graded result to HTML
        filestem = Path(notebook_path).name
        graded_html_path = notebook_path.replace('.ipynb', '-graded.html')
        html_exporter = HTMLExporter()
        r = html_exporter.from_notebook_node(nb, resources={
           'metadata': { 'name': filestem }
        })
        with open(graded_html_path, 'w', encoding="utf-8") as f:
            f.write(r[0])
        
        # LOCAL ENVIRONMENT ONLY
        # the Lambda handler only processes one file instead of
        # running a batch
        # the code below generates a CSV to show results for multiple files
        # get text summary of user's graded result
        text_summary = lambdagrader.generate_text_summary(graded_result)  
        result_summary = {
            'filename': graded_result['filename'],
            'grading_finished_at': graded_result['grading_finished_at'],
            'grading_duration_in_seconds': graded_result['grading_duration_in_seconds'],
            'learner_score': graded_result['learner_score'],
            'total_available': graded_result['total_available'],
            'num_test_cases': graded_result['num_test_cases'],
            'num_passed_cases': graded_result['num_passed_cases'],
            'num_failed_cases': graded_result['num_failed_cases'],
            'summary': text_summary,
            'test_cases_hash': graded_result['test_cases_hash'],
            'grader_python_version': graded_result['grader_python_version'],
            'grader_platform': graded_result['grader_platform']
        }
        graded_results.append(result_summary)
        
        print(f'Complete')
    except CellExecutionError as e:
        print(f'CellExecutionError on {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)

df_summary.to_csv(
    f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
    index=None
)

Grading test-notebooks/exercise-05\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb
Complete
Grading test-notebooks/exercise-05\esparzajoel_51499_7451315_Copy_of_exercise_05_pandas_filtering_sorting.ipynb
Complete
Grading test-notebooks/exercise-05\tabornatalie_8175_7404582_exercise_05_pandas_filtering_sorting.ipynb
Complete


In [22]:
from copydetect import CopyDetector
detector = CopyDetector(test_dirs=["test-notebooks/exercise-05"], extensions=["py"])
detector.run()
detector.generate_html_report()

  0.00: Generating file fingerprints


   100%|█████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 22.90it/s]


  0.27: Beginning code comparison


   100%|███████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2999.50it/s]


  0.27: Code comparison completed
Output saved to ./report.html


In [ ]:
# from lambdagrader import *

notebook_path = '../notebooks\\case-study-04-rideshare-trips-SOLUTION.ipynb'
nb = nbformat.read(notebook_path, as_version=4)

tcs = extract_test_cases_metadata_from_notebook(nb)

s = 0

for o in tcs:
    s += o['points']
    
print(s)